In [1]:
import sys
sys.path.append("../src")
from text2sql import hello
print(hello.message)

hello, world!


In [2]:
import os
import time

from dotenv import load_dotenv

from text2sql.engine.embeddings import EmbeddingResult, BedrockCohereEmbedder
from text2sql.engine.generation import GenerationResult, AzureGenerator, GCPGenerator

/home/derek/miniconda3/envs/bird/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
load_dotenv()

True

In [4]:
system_prompt = "you are a terse philosophy expert who responds in a concise manner."


user_template = """answer the question based on this excerpt from Bertrand Russell's 'The Problems of Philosophy':

```
CHAPTER IV. IDEALISM

The word 'idealism' is used by different philosophers in somewhat
different senses. We shall understand by it the doctrine that whatever
exists, or at any rate whatever can be known to exist, must be in
some sense mental. This doctrine, which is very widely held among
philosophers, has several forms, and is advocated on several different
grounds. The doctrine is so widely held, and so interesting in itself,
that even the briefest survey of philosophy must give some account of
it.

Those who are unaccustomed to philosophical speculation may be inclined
to dismiss such a doctrine as obviously absurd. There is no doubt that
common sense regards tables and chairs and the sun and moon and material
objects generally as something radically different from minds and the
contents of minds, and as having an existence which might continue if
minds ceased. We think of matter as having existed long before there
were any minds, and it is hard to think of it as a mere product of
mental activity. But whether true or false, idealism is not to be
dismissed as obviously absurd.

We have seen that, even if physical objects do have an independent
existence, they must differ very widely from sense-data, and can only
have a _correspondence_ with sense-data, in the same sort of way in
which a catalogue has a correspondence with the things catalogued. Hence
common sense leaves us completely in the dark as to the true intrinsic
nature of physical objects, and if there were good reason to regard them
as mental, we could not legitimately reject this opinion merely because
it strikes us as strange. The truth about physical objects _must_ be
strange. It may be unattainable, but if any philosopher believes that
he has attained it, the fact that what he offers as the truth is strange
ought not to be made a ground of objection to his opinion.

The grounds on which idealism is advocated are generally grounds derived
from the theory of knowledge, that is to say, from a discussion of the
conditions which things must satisfy in order that we may be able to
know them. The first serious attempt to establish idealism on such
grounds was that of Bishop Berkeley. He proved first, by arguments which
were largely valid, that our sense-data cannot be supposed to have an
existence independent of us, but must be, in part at least, 'in' the
mind, in the sense that their existence would not continue if there were
no seeing or hearing or touching or smelling or tasting. So far, his
contention was almost certainly valid, even if some of his arguments
were not so. But he went on to argue that sense-data were the only
things of whose existence our perceptions could assure us; and that
to be known is to be 'in' a mind, and therefore to be mental. Hence he
concluded that nothing can ever be known except what is in some mind,
and that whatever is known without being in my mind must be in some
other mind.

In order to understand his argument, it is necessary to understand his
use of the word 'idea'. He gives the name 'idea' to anything which
is _immediately_ known, as, for example, sense-data are known. Thus a
particular colour which we see is an idea; so is a voice which we hear,
and so on. But the term is not wholly confined to sense-data. There will
also be things remembered or imagined, for with such things also we have
immediate acquaintance at the moment of remembering or imagining. All
such immediate data he calls 'ideas'.

He then proceeds to consider common objects, such as a tree, for
instance. He shows that all we know immediately when we 'perceive' the
tree consists of ideas in his sense of the word, and he argues that
there is not the slightest ground for supposing that there is anything
real about the tree except what is perceived. Its being, he says,
consists in being perceived: in the Latin of the schoolmen its '_esse_'
is '_percipi_'. He fully admits that the tree must continue to exist
even when we shut our eyes or when no human being is near it. But this
continued existence, he says, is due to the fact that God continues to
perceive it; the 'real' tree, which corresponds to what we called the
physical object, consists of ideas in the mind of God, ideas more or
less like those we have when we see the tree, but differing in the fact
that they are permanent in God's mind so long as the tree continues
to exist. All our perceptions, according to him, consist in a
partial participation in God's perceptions, and it is because of this
participation that different people see more or less the same tree. Thus
apart from minds and their ideas there is nothing in the world, nor is
it possible that anything else should ever be known, since whatever is
known is necessarily an idea.

There are in this argument a good many fallacies which have been
important in the history of philosophy, and which it will be as well to
bring to light. In the first place, there is a confusion engendered by
the use of the word 'idea'. We think of an idea as essentially something
in somebody's mind, and thus when we are told that a tree consists
entirely of ideas, it is natural to suppose that, if so, the tree
must be entirely in minds. But the notion of being 'in' the mind is
ambiguous. We speak of bearing a person in mind, not meaning that the
person is in our minds, but that a thought of him is in our minds. When
a man says that some business he had to arrange went clean out of his
mind, he does not mean to imply that the business itself was ever in his
mind, but only that a thought of the business was formerly in his mind,
but afterwards ceased to be in his mind. And so when Berkeley says that
the tree must be in our minds if we can know it, all that he really has
a right to say is that a thought of the tree must be in our minds. To
argue that the tree itself must be in our minds is like arguing that a
person whom we bear in mind is himself in our minds. This confusion
may seem too gross to have been really committed by any competent
philosopher, but various attendant circumstances rendered it possible.
In order to see how it was possible, we must go more deeply into the
question as to the nature of ideas.
```

question:
{question}
"""

In [5]:
questions = [
"""Russell states that idealism "must not be dismissed as obviously absurd" even though it contradicts common sense. How might we reconcile philosophical positions that contradict our intuitive understanding of the world? Is common sense a reliable guide for philosophical inquiry?""",
"""Berkeley argues that "to be is to be perceived" (esse est percipi). Consider the implications of this view: if a tree falls in a forest with no one around to perceive it, does it make a sound? How might Berkeley respond to this classic philosophical question?""",
"""Russell identifies what he considers "fallacies" in Berkeley's argument about ideas and minds. Do you find Russell's criticism compelling? What distinguishes "being in the mind" from "having a thought about something"?""",
"""The excerpt suggests that our knowledge of physical objects is indirect and perhaps impossible to fully attain. How does this challenge affect our scientific understanding of the world? Can we have meaningful knowledge about things we cannot directly perceive?""",
"""Berkeley posits God as the continuous perceiver of objects when humans aren't perceiving them. What role does theology play in his idealist philosophy, and how might a modern philosopher approach this aspect of his argument?""",
]

In [6]:
embedder = BedrockCohereEmbedder(
    model=os.getenv("AWS_MODEL_NAME"),
    region_name=os.getenv("AWS_REGION_NAME"),
    input_type=os.getenv("AWS_INPUT_TYPE"),
    sleep_ms=0,
)


In [9]:
embedding_results = []
for question in questions:
    result = embedder.embed(question)
    assert type(result) is EmbeddingResult
    assert len(result.embedding) > 0
    assert result.input_characters > 0
    assert result.inf_time_ms > 0
    assert len(result.embedding) == 1024
    embedding_results.append(result)

In [10]:
embedding_results[0]

EmbeddingResult(embedding=[-0.006137848, 0.002954483, -0.005470276, 0.044647217, -0.025482178, -0.015594482, 0.017333984, -0.02406311, 0.0040779114, -0.025253296, -0.014625549, -0.041931152, 0.00024819374, 0.0072517395, -0.059692383, 0.019836426, -0.002506256, -0.00092315674, -0.020492554, -0.032440186, -0.014312744, 0.030685425, 0.0362854, 0.030288696, -0.0340271, 0.018463135, 0.02520752, -0.020004272, 0.03326416, 0.0262146, -0.00061130524, -0.014060974, 0.03302002, 0.0038852692, 0.016586304, 0.005832672, 0.0049858093, -0.04159546, -0.0041885376, 0.030319214, 0.03427124, 0.020889282, 0.011787415, 0.003900528, -0.0036907196, -0.0022583008, 0.003736496, -0.03186035, -0.04537964, 0.04208374, -0.0057296753, -0.0039634705, 0.018814087, -0.0012950897, 0.021575928, 0.013587952, 0.02458191, -0.023635864, -0.013557434, -0.025375366, -0.0009059906, -0.03286743, -0.042755127, 0.025848389, -0.044189453, 0.004508972, 0.0043182373, -0.059539795, -0.009002686, 0.0022068024, -0.018493652, -0.00944519

In [11]:
gcp_generator = GCPGenerator(
    model="gemini-1.5-flash",
    api_key=os.getenv("GCP_KEY"),
)

In [12]:
gcp_responses = []
for question in questions:
    test_messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_template.format(question=question)}
    ]
    start_time = time.time()
    response = gcp_generator.generate(test_messages, temperature=0.0)
    end_time = time.time()
    inf_time_ms = int((end_time - start_time) * 1000)
    assert type(response) is GenerationResult
    assert response.tokens.prompt_tokens > 0
    assert response.tokens.output_tokens > 0
    assert response.text
    assert (inf_time_ms-500) < response.inf_time_ms < (inf_time_ms+500)
    gcp_responses.append(response)

In [13]:
# print token info for each response
for response in gcp_responses:
    print(response.tokens.cached_tokens, response.tokens.prompt_tokens, response.tokens.output_tokens, response.tokens.total_tokens)


48 1478 48 1526
11 1489 11 1500
57 1477 57 1534
55 1475 55 1530
45 1476 45 1521


In [14]:
# print question and response text
for idx, response in enumerate(gcp_responses):
    print(questions[idx])
    print(response.text)
    print()


Russell states that idealism "must not be dismissed as obviously absurd" even though it contradicts common sense. How might we reconcile philosophical positions that contradict our intuitive understanding of the world? Is common sense a reliable guide for philosophical inquiry?
Idealism's strangeness doesn't invalidate it.  Reconciliation requires rigorous analysis of concepts and arguments, not reliance on intuition. Common sense is unreliable in philosophical inquiry; it's a starting point, not a conclusion.


Berkeley argues that "to be is to be perceived" (esse est percipi). Consider the implications of this view: if a tree falls in a forest with no one around to perceive it, does it make a sound? How might Berkeley respond to this classic philosophical question?
No sound.  Berkeley: God perceives it.


Russell identifies what he considers "fallacies" in Berkeley's argument about ideas and minds. Do you find Russell's criticism compelling? What distinguishes "being in the mind" fro

In [15]:
gcp_responses[0]

GenerationResult(text="Idealism's strangeness doesn't invalidate it.  Reconciliation requires rigorous analysis of concepts and arguments, not reliance on intuition. Common sense is unreliable in philosophical inquiry; it's a starting point, not a conclusion.\n", tokens=TokenUsage(cached_tokens=48, prompt_tokens=1478, output_tokens=48, total_tokens=1526), inf_time_ms=6015, status='ok')

In [27]:
azure_generator = AzureGenerator(
    model="gpt-4o-mini-dev",
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    azure_endpoint=os.getenv("AZURE_OPENAI_API_ENDPOINT"),
    api_version=os.getenv("AZURE_OPENAI_API_VERSION"),
)

In [32]:
azure_responses = []
for question in questions:
    test_messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_template.format(question=question)}
    ]
    start_time = time.time()
    response = azure_generator.generate(test_messages, temperature=0.0)
    end_time = time.time()
    inf_time_ms = int((end_time - start_time) * 1000)
    assert type(response) is GenerationResult
    assert response.tokens.prompt_tokens > 0
    assert response.tokens.output_tokens > 0
    assert response.text
    assert (inf_time_ms-500) < response.inf_time_ms < (inf_time_ms+500)
    azure_responses.append(response)


In [ ]:
# print token info for each response
for response in azure_responses:
    print(response.tokens.cached_tokens, response.tokens.prompt_tokens, response.tokens.output_tokens, response.tokens.total_tokens)


In [ ]:
# print question and response text
for idx, response in enumerate(azure_responses):
    print(questions[idx])
    print(response.text)
    print()
